In [141]:
!pip install PyWavelets

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [142]:
!cp "/content/drive/MyDrive/CS5100 Project/resized_images_v4_224.zip" -r "/content/resized_images_v4_224.zip"
!unzip /content/resized_images_v4_224.zip -d /content/resized_images_v4_224

Streaming output truncated to the last 5000 lines.
  inflating: /content/resized_images_v4_224/resized_images_v4_224/00028897_014.png  
  inflating: /content/resized_images_v4_224/resized_images_v4_224/00028897_015.png  
  inflating: /content/resized_images_v4_224/resized_images_v4_224/00028897_016.png  
  inflating: /content/resized_images_v4_224/resized_images_v4_224/00028897_017.png  
  inflating: /content/resized_images_v4_224/resized_images_v4_224/00028897_018.png  
  inflating: /content/resized_images_v4_224/resized_images_v4_224/00028897_019.png  
  inflating: /content/resized_images_v4_224/resized_images_v4_224/00028897_020.png  
  inflating: /content/resized_images_v4_224/resized_images_v4_224/00028898_000.png  
  inflating: /content/resized_images_v4_224/resized_images_v4_224/00028899_000.png  
  inflating: /content/resized_images_v4_224/resized_images_v4_224/00028899_001.png  
  inflating: /content/resized_images_v4_224/resized_images_v4_224/00028899_002.png  
  inflating: /

In [143]:
import pickle
def read_pickle_file(filepath):
  with open(filepath, "rb") as f:
    return pickle.load(f)

In [144]:
filepath = "/content/class_dict_wo_NF.pkl"
class_dict = read_pickle_file(filepath)

In [145]:
df = pd.read_csv("/content/final_df_all_images_wo_NF_labels.csv")

In [146]:
df.drop(columns=["disease_vec"], axis=1, inplace=True)
df['disease_vec'] = df.apply(lambda x: [x[class_dict.values()].values], 1).map(lambda x: x[0])
df["disease_vec"] = df["disease_vec"].apply(lambda x: np.array(x, dtype=int))

In [147]:
all_labels = list(class_dict.values())
# all_labels
total_data = 0
for c_label in all_labels:
  print("{}: {}".format(c_label, df[c_label].sum()))
  total_data += df[c_label].sum()

print("\n")
print("Total Size of Data: {}".format(total_data))

Atelectasis: 11559
Cardiomegaly: 2776
Consolidation: 4667
Edema: 2303
Effusion: 13317
Emphysema: 2516
Fibrosis: 1686
Hernia: 227
Infiltration: 19894
Mass: 5782
Nodule: 6331
Pleural_Thickening: 3385
Pneumonia: 1431
Pneumothorax: 5302


Total Size of Data: 81176


In [148]:
df.head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11,Atelectasis,Cardiomegaly,Consolidation,Edema,Effusion,Emphysema,Fibrosis,Hernia,Infiltration,Mass,Nodule,Pleural_Thickening,Pneumonia,Pneumothorax,disease_vec
0,00000001_000.png,['Cardiomegaly'],0,1,58,M,PA,2682,2749,0.143,0.143,NaN,0,1,0,0,0,0,0,0,0,0,0,0,0,0,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,00000001_001.png,"['Cardiomegaly', 'Emphysema']",1,1,58,M,PA,2894,2729,0.143,0.143,NaN,0,1,0,0,0,1,0,0,0,0,0,0,0,0,"[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]"
2,00000001_002.png,"['Cardiomegaly', 'Effusion']",2,1,58,M,PA,2500,2048,0.168,0.168,NaN,0,1,0,0,1,0,0,0,0,0,0,0,0,0,"[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,00000002_000.png,['No Finding'],0,2,81,M,PA,2500,2048,0.171,0.171,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,00000003_000.png,['Hernia'],0,3,81,F,PA,2582,2991,0.143,0.143,NaN,0,0,0,0,0,0,0,1,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]"


In [149]:
df["image_path"] = "/content/resized_images_v4_224/resized_images_v4_224/" + df["Image Index"]

In [150]:
X, y = df["image_path"], df["disease_vec"]

In [151]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101, shuffle=True)
X_train, X_test = pd.DataFrame(X_train), pd.DataFrame(X_test)
train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_test, y_test], axis=1).reset_index().drop("index", axis=1)

rs = np.random.RandomState(101)
msk = rs.rand(len(train_df)) < 0.85
valid_df = train_df[~msk].reset_index().drop("index", axis=1)
train_df = train_df[msk].reset_index().drop("index", axis=1)

In [152]:
# HOG - Histogram of Oriented Gradients
from skimage.io import imread
from skimage.transform import resize
from skimage.feature import hog
from skimage import exposure
import matplotlib.pyplot as plt
from skimage.color import rgb2gray

def hog_features(path):
  # reading the image
  img = imread(path)
  img = rgb2gray(img)
  resized_img = resize(img, (224,224))
  exposure_balanced_image = exposure.rescale_intensity(resized_img, in_range=(0, 225), out_range=(0, 225))
  fd = hog(exposure_balanced_image, orientations=4, pixels_per_cell=(8, 8),
                	cells_per_block=(1, 1), visualize=False, multichannel=False)
  return (fd.tolist() , exposure_balanced_image)

In [153]:
# LBP- Local Binary Pattern
from skimage.feature import local_binary_pattern
from skimage import data

def LBP(exposure_balanced_image, radius = 3):

  def hist(lbp):
    n_bins = int(lbp.max() + 1)
    return plt.hist(lbp.ravel(), density=True, bins=n_bins, range=(0, n_bins),facecolor='0.5')

  n_points = 8 * radius
  lbp = local_binary_pattern(exposure_balanced_image, n_points, radius, method = 'uniform')
  histogram = hist(lbp)
  plt.close()
  histogram = histogram[0].tolist()
  return histogram

In [154]:
# Gray level co-occurance matrix

import numpy as np
from skimage.feature import greycomatrix, greycoprops
from skimage import io, color, img_as_ubyte

def GLCM(image):
  image = img_as_ubyte(image)

  bins = np.array([0, 16, 32, 48, 64, 80, 96, 112, 128, 144, 160, 176, 192, 208, 224, 240, 255]) #16-bit
  inds = np.digitize(image, bins)

  max_value = inds.max()+1
  matrix_coocurrence = greycomatrix(inds, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4], levels=max_value, normed=False, symmetric=False)

  # GLCM properties
  def contrast_feature(matrix_coocurrence):
      contrast = greycoprops(matrix_coocurrence, 'contrast')
      return contrast

  def dissimilarity_feature(matrix_coocurrence):
      dissimilarity = greycoprops(matrix_coocurrence, 'dissimilarity')    
      return dissimilarity

  def homogeneity_feature(matrix_coocurrence):
      homogeneity = greycoprops(matrix_coocurrence, 'homogeneity')
      return homogeneity

  def energy_feature(matrix_coocurrence):
      energy = greycoprops(matrix_coocurrence, 'energy')
      return energy

  def correlation_feature(matrix_coocurrence):
      correlation = greycoprops(matrix_coocurrence, 'correlation')
      return correlation

  def entropy_feature(matrix_coocurrence):
      entropy = greycoprops(matrix_coocurrence, 'entropy')
      return entropy

  # print(contrast_feature(matrix_coocurrence))
  # print(dissimilarity_feature(matrix_coocurrence))
  # print(homogeneity_feature(matrix_coocurrence))
  # print(energy_feature(matrix_coocurrence))
  # print(correlation_feature(matrix_coocurrence))
  return np.concatenate((contrast_feature(matrix_coocurrence),dissimilarity_feature(matrix_coocurrence),homogeneity_feature(matrix_coocurrence),energy_feature(matrix_coocurrence),correlation_feature(matrix_coocurrence)), axis = 1)


In [155]:
import pywt
import pywt.data
def wavelet_transform(image):

  coef = pywt.wavedec2(image, 'db1' , level=4) # level3 decomposition #use db1 or haar

  cA = coef[0]
  (cH1,cV1,cD1) =  coef[-1]#level1
  (cH2,cV2,cD2) =  coef[-2]#level2
  (cH3,cV3,cD3) =  coef[-3]#level3
  (cH4,cV4,cD4) =  coef[-4]#level4 output of level4 decomp and furthur level dont offer better features
  return cA.ravel().tolist()+cH4.ravel().tolist()+cV4.ravel().tolist()+cD4.ravel().tolist()


In [156]:
path = '/content/00000075_000.png'
fd, exposure_balanced_image = hog_features(path)
hist = LBP(exposure_balanced_image)
covariance_matrix = GLCM(exposure_balanced_image).tolist()[0]
wt = wavelet_transform(exposure_balanced_image)
lst = fd + covariance_matrix + hist + wt
df = pd.DataFrame(lst)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
  if sys.path[0] == '':


In [159]:
def extract_features(dataset):
  return_df = pd.DataFrame()
  for i in range(dataset.shape[0]):
    lst = list()
    path = dataset['image_path'][i]
    fd, exposure_balanced_image = hog_features(path)
    hist = LBP(exposure_balanced_image)
    covariance_matrix = GLCM(exposure_balanced_image).tolist()[0]
    wt = wavelet_transform(exposure_balanced_image)
    lst = fd + covariance_matrix + hist 
    df = pd.DataFrame(lst)
    return_df = return_df.append(df)
  
  return_df['disease_vec'] = dataset['disease_vec']
  return return_df


In [ ]:
train_df = extract_features(train_df)
test_df = extract_features(test_df)
valid_df = extract_features(valid_df)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: FutureWarning: Non RGB image conversion is now deprecated. For RGBA images, please use rgb2gray(rgba2rgb(rgb)) instead. In version 0.19, a ValueError will be raised if input image last dimension length is not 3.
  if sys.path[0] == '':


In [ ]:
with open("/content/drive/MyDrive/CS5100 Project/manual_features.pkl", "wb") as fp:
  pickle.dump([train_df,test_df,val_df], fp)

In [ ]:
train_df.head()